_Lambda School Data Science — Tree Ensembles_ 

# Decision Trees Assignment

## Part 1: House Price Regression

Apply decision trees to the Ames housing dataset you've worked with this week!

- Try multiple features
- Try features you've engineered
- Try different `max_depth` paramaters
- What's the best Test Root Mean Squared Error you can get? *Share with your cohort on Slack!*
- What's a cool visualization you can make? *Share with your cohort on Slack!*

In [175]:
# import libraries
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

# set pandas view options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

Ryan's Office Hours made me realize that my data cleaning process and feature selection is problematic by being performed prior to running the regression. 

I'm going to remedy this by creating a datacleaning and feature selection function that should prevent cross contamination.

In [179]:
# import dataset

url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv'
df = pd.read_csv(url)
df.head().T    

,0,1,2,3,4
Id,1,2,3,4,5
MSSubClass,60,20,60,70,60
MSZoning,RL,RL,RL,RL,RL
LotFrontage,65,80,68,60,84
LotArea,8450,9600,11250,9550,14260
Street,Pave,Pave,Pave,Pave,Pave
Alley,NaN,NaN,NaN,NaN,NaN
LotShape,Reg,Reg,IR1,IR1,IR1
LandContour,Lvl,Lvl,Lvl,Lvl,Lvl
Utilities,AllPub,AllPub,AllPub,AllPub,AllPub


In [166]:
# data cleaning function

# ok, this isn't working, I'm giving up

def data_cleaning(data):
    # drop columns that seem unimportant and NaNy
    data = data.drop(columns = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
                                'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
                                'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 
                                'Fence', 'MiscFeature', 'Id'], axis=1)

    # fill with 0s because these categories probably mean 0 instead of nan
    data['LotFrontage'] = data['LotFrontage'].fillna(0)
    data['Alley'] = data['Alley'].fillna(0)
    
    # and encode categorical variables
    for col in data.columns:
        if data[col].dtype=='O':
            data[col] = pd.Categorical(data[col])
            data[col] = data[col].cat.codes
    
    # resolve a float64 error with Linear Regression
    data = data.fillna(method='ffill')
    for col in data.columns:
        if data[col].dtype=='float64':
            data[col] = (data[col]).astype(np.int64)
    
    # remove outliers
    data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]

    return data
        
    

    

In [167]:
# Now I'll create features in a feature function 

# did not work, will feature manually

def feature_engineer(data):
    
    # first I'm going to engineer a number to represent year since remodelled
    data['Remod_Age'] = data['YrSold'] - data['YearRemodAdd']
    # and create a polynomial remod_age feature to account for depreciation
    data['Remod_Age_squared'] = data['Remod_Age'] ** 2
    # I will also add a total sqf and a polynomial companion variable
    data['Total_SF'] = data['TotalBsmtSF'] + data['GrLivArea']
    data['Total_SF_squared'] = data['Total_SF']**2
    # total porch and a polynomial companion variable
    data['all_porch'] = data['WoodDeckSF']+ data['OpenPorchSF'] + data['EnclosedPorch'] + data['ScreenPorch']
    data['all_porch_squared'] = data['all_porch']**2
    return data

In [168]:
# and apply our first two models in the lin model function

def run_linear_model(X, y):
    # Split into test and train data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=42)
    
    #clean data: #this function failed so I'm commenting it
    data_cleaning(X_train)
    data_cleaning(X_test)

    
    #engineer features #this function failed so I'm commenting it
#     feature_engineer(X_train)
#     feature_engineer(X_test)
#     feature_engineer(y_train)
#     feature_engineer(y_test)
    
    # Fit model using train data
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions using test features
    y_pred = model.predict(X_test)

    # Compare predictions to test target
    rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
    r2 = r2_score(y_test, y_pred)

    print('Root Mean Squared Error', rmse)
    print('R^2 Score', r2)
    print('Intercept', model.intercept_)
    coefficients = pd.Series(model.coef_, X_train.columns)
    print(coefficients.to_string())

In [169]:
# I'm running a linear model first to check clean import
features = df.columns.drop(['SalePrice'])
target = 'SalePrice'

y = df[target]
X = df[features]

run_linear_model(X, y)
# X
# y

/usr/lib/python3/dist-packages/scipy/stats/stats.py:2247: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
/usr/lib/python3/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less


ValueError: could not convert string to float: 'Normal'

In [177]:
# manual data cleaning
df = df.drop(['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
              'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
              'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 
              'Fence', 'MiscFeature', 'Id'], axis=1)

# fill with 0s because these categories probably mean 0 instead of nan
df['LotFrontage'] = df['LotFrontage'].fillna(0)
df['Alley'] = df['Alley'].fillna(0)

# and encode categorical variables
for col in df.columns:
      if df[col].dtype=='O':
        df[col] = pd.Categorical(df[col])
        df[col] = df[col].cat.codes
# resolve a float64 error with Linear Regression
df = df.fillna(method='ffill')
for col in df_coded.columns:
    if df[col].dtype=='float64':
        df[col] = (df[col]).astype(np.int64)
# remove outliers
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

# first I'm going to engineer a number to represent year since remodelled
df['Remod_Age'] = df['YrSold'] - df['YearRemodAdd']
# and create a polynomial remod_age feature to account for depreciation
df['Remod_Age_squared'] = df['Remod_Age'] ** 2
# I will also add a total sqf and a polynomial companion variable
df['Total_SF'] = df['TotalBsmtSF'] + df['GrLivArea']
df['Total_SF_squared'] = df['Total_SF']**2
# total porch and a polynomial companion variable
df['all_porch'] = df['WoodDeckSF']+ df['OpenPorchSF'] + df['EnclosedPorch'] + df['ScreenPorch']
df['all_porch_squared'] = df['all_porch']**2

In [178]:
df['SaleCondition']

0       4
2       4
4       4
6       4
10      4
12      4
13      5
14      4
16      4
20      5
22      4
23      4
25      4
27      4
31      4
32      4
35      4
36      4
42      4
43      4
49      4
57      4
62      4
65      4
67      4
68      4
71      4
73      4
75      4
79      4
81      4
82      4
84      4
85      4
86      4
89      4
92      4
94      4
96      4
97      4
99      4
100     4
101     4
103     4
107     5
110     4
111     4
112     5
117     5
119     5
       ..
1348    4
1351    4
1354    4
1355    4
1356    4
1357    4
1359    4
1362    4
1363    5
1367    4
1368    4
1370    4
1372    4
1375    5
1377    4
1378    4
1381    4
1384    4
1390    4
1392    4
1394    5
1395    4
1399    4
1400    4
1401    4
1402    5
1404    3
1406    4
1407    4
1408    4
1409    4
1410    4
1411    4
1420    4
1421    4
1422    4
1425    4
1426    4
1429    4
1431    4
1432    4
1433    4
1436    4
1441    4
1444    4
1446    4
1447    4
1451    5
1452    4


In [129]:
# VIF for fun
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = [variance_inflation_factor((sm.add_constant(X)).values, i) for i in range(len(X.columns))]

pd.Series(vif, X.columns)

/usr/lib/python3/dist-packages/numpy/core/fromnumeric.py:57: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return getattr(obj, method)(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1543: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.6/dist-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


MSSubClass           16.377850
MSZoning              1.950719
LotFrontage           1.473333
LotArea               2.527985
Street                0.000000
Alley                 1.463576
LotShape              1.506292
LandContour           1.218627
Utilities                  NaN
LotConfig             1.233822
LandSlope                  NaN
Neighborhood          1.326930
Condition1            1.270185
Condition2            0.000000
BldgType             12.256962
HouseStyle            5.216832
OverallQual           4.027726
OverallCond           2.008912
YearBuilt             8.680034
YearRemodAdd               inf
RoofStyle             1.272461
RoofMatl              0.000000
Exterior1st           6.884421
Exterior2nd           6.703921
MasVnrType            1.412685
MasVnrArea            1.704825
ExterQual             2.945660
ExterCond             1.241689
Foundation            3.007890
BsmtFinSF1                 inf
BsmtFinSF2                 inf
BsmtUnfSF                  inf
TotalBsm

In [120]:
tree = DecisionTreeRegressor()
tree.fit(X, y)
print('R^2', tree.score(X, y))

R^2 0.999984948138


In [135]:
# now i'll make a decision tree version of my function:
def run_tree_model(X, y, max_depth, min_leaf):
    # Split into test and train data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=42)
    
    #clean data: #this function failed so I'm commenting it
#     data_cleaning(X_train)
#     data_cleaning(X_test)
#     data_cleaning(y_train)
#     data_cleaning(y_test)
    
    #engineer features #this function failed so I'm commenting it
#     feature_engineer(X_train)
#     feature_engineer(X_test)
#     feature_engineer(y_train)
#     feature_engineer(y_test)
    for deep in max_depth:
        for leaf in min_leaf:
            # Fit model using train data
            model = DecisionTreeRegressor(max_depth=deep, min_samples_leaf=leaf)
            model.fit(X_train, y_train)

            # Make predictions using test features
            y_pred = model.predict(X_test)

            # Compare predictions to test target
            rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
            r2 = tree.score(X, y)
            print('For max_depth = %s and min_samples_leaf = %s' % (deep,leaf))
            print('Root Mean Squared Error', rmse)
            print('R^2 Score', r2)


In [137]:
depth = [19,20,21,22]
leaves = [5,6,7,8,9,10]

run_tree_model(X, y, depth, leaves)

For max_depth = 19 and min_samples_leaf = 5
Root Mean Squared Error 29259.8382656
R^2 Score 0.999984948138
For max_depth = 19 and min_samples_leaf = 6
Root Mean Squared Error 30475.2036473
R^2 Score 0.999984948138
For max_depth = 19 and min_samples_leaf = 7
Root Mean Squared Error 29772.7117562
R^2 Score 0.999984948138
For max_depth = 19 and min_samples_leaf = 8
Root Mean Squared Error 29447.5456866
R^2 Score 0.999984948138
For max_depth = 19 and min_samples_leaf = 9
Root Mean Squared Error 29097.5151363
R^2 Score 0.999984948138
For max_depth = 19 and min_samples_leaf = 10
Root Mean Squared Error 28672.0797155
R^2 Score 0.999984948138
For max_depth = 20 and min_samples_leaf = 5
Root Mean Squared Error 29259.8382656
R^2 Score 0.999984948138
For max_depth = 20 and min_samples_leaf = 6
Root Mean Squared Error 30475.2036473
R^2 Score 0.999984948138
For max_depth = 20 and min_samples_leaf = 7
Root Mean Squared Error 29772.7117562
R^2 Score 0.999984948138
For max_depth = 20 and min_samples_l

## Part 2 / Stretch: "Play Tennis" Classification

We'll reproduce the "Play Tennis" example from Ross Quinlan's 1986 paper, [Induction of Decison Trees](https://link.springer.com/content/pdf/10.1007%2FBF00116251.pdf).

[According to Wikipedia](https://en.wikipedia.org/wiki/Ross_Quinlan), "John Ross Quinlan is a computer science researcher in data mining and decision theory. He has contributed extensively to the development of decision tree algorithms, including inventing the canonical C4.5 and ID3 algorithms."

#### "Table 1 shows a small training set"

In [32]:
import pandas as pd

columns = 'No. Outlook Temperature Humidity Windy PlayTennis'.split()

raw = """1 sunny hot high false N
2 sunny hot high true N
3 overcast hot high false P
4 rain mild high false P
5 rain cool normal false P
6 rain cool normal true N
7 overcast cool normal true P
8 sunny mild high false N
9 sunny cool normal false P
10 rain mild normal false P
11 sunny mild normal true P
12 overcast mild high true P
13 overcast hot normal false P
14 rain mild high true N"""

data = [row.split() for row in raw.split('\n')]
tennis = pd.DataFrame(data=data, columns=columns).set_index('No.')
tennis['PlayTennis'] = (tennis['PlayTennis'] == 'P').astype(int)

tennis

,Outlook,Temperature,Humidity,Windy,PlayTennis
No.,,,,,
1,sunny,hot,high,false,0
2,sunny,hot,high,true,0
3,overcast,hot,high,false,1
4,rain,mild,high,false,1
5,rain,cool,normal,false,1
6,rain,cool,normal,true,0
7,overcast,cool,normal,true,1
8,sunny,mild,high,false,0
9,sunny,cool,normal,false,1


#### "A decision tree that correctly classifies each object in the training set is given in Figure 2."

<img src="https://i.imgur.com/RD7d0u0.png" height="300">

In this dataset, the tennis player decided to play on 64% of the days, and decided not to on 36% of the days.

In [39]:
tennis['PlayTennis'].value_counts(normalize=True) * 100

1    64.285714
0    35.714286
Name: PlayTennis, dtype: float64

The tennis player played on 100% of the overcast days, 40% of the sunny days, and 60% of the  rainy days

In [38]:
tennis.groupby('Outlook')['PlayTennis'].mean() * 100

Outlook
overcast    100.0
rain         60.0
sunny        40.0
Name: PlayTennis, dtype: float64

On sunny days, the tennis player's decision depends on the humidity. (The Outlook and Humidity features interact.)

In [40]:
sunny = tennis[tennis['Outlook']=='sunny']
sunny.groupby('Humidity')['PlayTennis'].mean() * 100

Humidity
high        0
normal    100
Name: PlayTennis, dtype: int64

On rainy days, the tennis player's decision depends on the wind. (The Outlook and Windy features interact.)

In [41]:
rainy = tennis[tennis['Outlook']=='rain']
rainy.groupby('Windy')['PlayTennis'].mean() * 100

Windy
false    100
true       0
Name: PlayTennis, dtype: int64

#### Before modeling, we will ["encode" categorical variables, using pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html?highlight=get_dummies#computing-indicator-dummy-variables)

In [45]:
y = tennis['PlayTennis']
X = pd.get_dummies(tennis.drop(columns='PlayTennis'))
X

(14, 10) (14,)


,Outlook_overcast,Outlook_rain,Outlook_sunny,Temperature_cool,Temperature_hot,Temperature_mild,Humidity_high,Humidity_normal,Windy_false,Windy_true
No.,,,,,,,,,,
1,0,0,1,0,1,0,1,0,1,0
2,0,0,1,0,1,0,1,0,0,1
3,1,0,0,0,1,0,1,0,1,0
4,0,1,0,0,0,1,1,0,1,0
5,0,1,0,1,0,0,0,1,1,0
6,0,1,0,1,0,0,0,1,0,1
7,1,0,0,1,0,0,0,1,0,1
8,0,0,1,0,0,1,1,0,1,0
9,0,0,1,1,0,0,0,1,1,0


## Train a Decision Tree Classifier
Get a score of 100% (accuracy)

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

## Compare to Logistic Regression
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

## Visualize the tree
https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html